In [54]:
from dotenv import load_dotenv, find_dotenv
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import CSVLoader
from langchain_community.vectorstores import Chroma
from langchain_openai.embeddings import OpenAIEmbeddings
import os
from datetime import datetime
from modules.set_model import llm_model
from langchain.callbacks import get_openai_callback
from modules.evaluation import generate_qas, evaluate
from modules.results_data import ResultsData

def vct_db_filename_gen(file_path):
    # Derive vector DB filename from CSV filename
    base_name = os.path.basename(file_path)
    db_file_name = os.path.splitext(base_name)[0] + ".vecdb"

    return os.path.join(os.path.dirname(file_path), db_file_name)

def check_and_load_vector_db(file_path, embedding):
    """
    Checks if a vector db file exists for the given file_path, 
    loads it if exists, otherwise creates it from the csv and saves it.
    """
    # Derive vector DB filename from CSV filename
    db_file_path = vct_db_filename_gen(file_path)

    # Check if the vector DB file exists
    if os.path.exists(db_file_path):
        print(f"Loading existing vector DB from {db_file_path}")
        db = Chroma(persist_directory=db_file_path, embedding_function=embedding)
    else:
        print(f"Vector DB not found. Creating from {file_path}")
        # Load the CSV and create the vector DB
        loader = CSVLoader(file_path=file_path)
        documents = loader.load()
        # Save the newly created vector DB
        db = Chroma.from_documents(documents, embedding, persist_directory=db_file_path)
        print(f"Saved new vector DB to {db_file_path}")
    
    return db

def write_markdown_table_to_file(markdown_table, file_name):
    """
    Writes the given markdown table string to a file.

    Parameters:
    - markdown_table: A string containing the markdown table to write to the file.
    - file_name: The name of the file to write the markdown table to.
    """
    with open(file_name, 'w') as file:
        file.write(markdown_table)

    # Specify the file name
    file_name = 'results_table.md'

    # Write the markdown table to the file
    write_markdown_table_to_file(markdown_table, file_name)

In [49]:
class ResultsData:
    def __init__(self, chain_type, time=None, tokens_used=None, example_number=None, predicted_query=None, predicted_answer=None, answer=None, result=None):
        self.chain_type = chain_type
        self.eval = []
        if example_number is not None:
            self.append_evaluation(time, tokens_used, example_number, predicted_query, answer, predicted_answer, result)
    
    def append_evaluation(self, time, tokens_used, example_number, predicted_query, answer, predicted_answer=None, result=None):
        """Append a new evaluation result to the eval list."""
        self.eval.append({
            "time": time,
            "tokens_used": tokens_used,
            "example_number": example_number,
            "query": predicted_query,
            "predicted_answer": predicted_answer,
            "answer": answer,
            "result": result
        })


In [39]:
# Basic Setup
_ = load_dotenv(find_dotenv()) # read local .env file
results_data = []
strategies = ["stuff", "map_reduce", "refine", "map_rerank"]

# Load data into vector db or use existing one
file_path = '../data/OutdoorClothingCatalog_1000.csv'
embedding = OpenAIEmbeddings()  # Define embedding

# Check if vector DB exists for the CSV, and load or create accordingly
db = check_and_load_vector_db(file_path, embedding)

queries = ["Please suggest a shirt with sunblocking", "Please suggest a shirt with sunblocking and tell me why this one", "Please suggest three shirts with sunblocking and tell me why. Give this back to me in markdown code as a table", "Please suggest three shirts with sunblocking and tell me why. Give this back to me in markdown code as a table, with a summary below outlining why sunblocking is important"]

    # Configure LLM for querying
    # layers vector db on llm to inform decisions and responses
llm = ChatOpenAI(temperature = 0.0, model=llm_model())
retriever = db.as_retriever()

Loading existing vector DB from ../data/OutdoorClothingCatalog_1000.vecdb


In [51]:
def qa_analysis(llm, chain_type, retriever, verbose, query, number, results_data):
    """
    Initializes a QA analysis with a given language model, chain type, and retriever.
    Then, it runs the QA analysis, timing its execution and printing the response along with the execution time.
    """
    # Initialize the RetrievalQA object with the specified parameters.
    qa = RetrievalQA.from_chain_type(
        llm=llm, 
        chain_type=chain_type, 
        retriever=retriever, 
        verbose=verbose
    )

    # Measure number of tokens used
    with get_openai_callback() as cb:
        start = datetime.now()

        try:
            # Execute the QA analysis
            response = qa.invoke(query)
        except ValueError as e: 
            response = e

        end = datetime.now()
    
    tokens_used = cb.total_tokens

    # Calculate the difference between the end and start timestamps to get the execution duration.
    # The duration is converted to milliseconds for a more precise and readable format.
    td = (end - start).total_seconds() * 10**3
    
    print(f"Response: {response}\nThe time of execution of above program is : {td:.03f}ms")

    found = False
    for item in results_data:
        if item.chain_type == chain_type:
            # Update the existing dictionary
            item.append_evaluation(time=td, tokens_used=tokens_used, example_number=number, 
                         predicted_query=query, answer=response)
            found = True
            break

    if not found:
        # Append a new instance of ResultsData if no matching chain_type was found
        results_data.append(ResultsData(chain_type=chain_type, time=td, tokens_used=tokens_used, 
                                        example_number=number, predicted_query=query, answer=response))

    print("\n\nTESTING\n:" + '\n'.join([str(item) for item in results_data]))

    return results_data

In [50]:
results_data = []

In [52]:
    # Manual analysis
for index, query in enumerate(queries, start=1):
        results_data = qa_analysis(llm, "stuff", retriever, True, query, index, results_data)
        results_data = qa_analysis(llm, "map_reduce", retriever, True, query, index, results_data)
        results_data = qa_analysis(llm, "refine", retriever, True, query, index, results_data)
        results_data = qa_analysis(llm, "map_rerank", retriever, True, query, index, results_data)




> Entering new RetrievalQA chain...

> Finished chain.
Response: {'query': 'Please suggest a shirt with sunblocking', 'result': "We have several options for shirts with sunblocking. Here are a few:\n\n1. Sun Shield Shirt: This high-performance sun shirt is guaranteed to protect from harmful UV rays. It is rated UPF 50+ and blocks 98% of the sun's harmful rays. It is made of 78% nylon and 22% Lycra Xtra Life fiber and is handwashable. \n\n2. Men's Plaid Tropic Shirt, Short-Sleeve: This shirt is rated UPF 50+ and blocks 98% of the sun's harmful UV rays. It is made of 52% polyester and 48% nylon and is machine washable. \n\n3. Men's TropicVibe Shirt, Short-Sleeve: This shirt has built-in UPF 50+ and is made of 71% Nylon and 29% Polyester. It is machine washable and wrinkle-resistant. \n\n4. Men's Tropical Plaid Short-Sleeve Shirt: This shirt is rated UPF 50+ and blocks 98% of the sun's harmful rays. It is made of 100% polyester and is wrinkle-resistant. \n\nAll of these shirts provide e

/opt/homebrew/lib/python3.11/site-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(



> Finished chain.
Response: {'query': 'Please suggest a shirt with sunblocking', 'result': "Men's Plaid Tropic Shirt, Short-Sleeve"}
The time of execution of above program is : 3448.791ms


TESTING
:<__main__.ResultsData object at 0x1569891d0>


> Entering new RetrievalQA chain...

> Finished chain.
Response: {'query': 'Please suggest a shirt with sunblocking and tell me why this one', 'result': 'I would suggest the Sun Shield Shirt as it is designed to block harmful UV rays with a UPF 50+ rating, which is the highest rated sun protection possible. The fabric is made of 78% nylon and 22% Lycra Xtra Life fiber, which is abrasion-resistant and wicks moisture for quick-drying comfort. Additionally, it fits comfortably over your favorite swimsuit and falls at the hip with a slightly fitted shape. The Skin Cancer Foundation recommends this fabric as an effective UV protectant.'}
The time of execution of above program is : 2249.480ms


TESTING
:<__main__.ResultsData object at 0x1569891d0>



/opt/homebrew/lib/python3.11/site-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(



> Finished chain.
Response: {'query': 'Please suggest a shirt with sunblocking and tell me why this one', 'result': "The Sun Shield Shirt provides UPF 50+ sun protection, blocking 98% of the sun's harmful rays. It is recommended by The Skin Cancer Foundation as an effective UV protectant. The fabric is made of 78% nylon and 22% Lycra Xtra Life fiber, and is abrasion resistant for season after season of wear. It also wicks moisture for quick-drying comfort and fits comfortably over your favorite swimsuit. "}
The time of execution of above program is : 6237.524ms


TESTING
:<__main__.ResultsData object at 0x1569891d0>


> Entering new RetrievalQA chain...

> Finished chain.
Response: {'query': 'Please suggest three shirts with sunblocking and tell me why. Give this back to me in markdown code as a table', 'result': "| Shirt Name | Sun Protection | Material | Additional Features |\n| --- | --- | --- | --- |\n| Men's Plaid Tropic Shirt, Short-Sleeve | UPF 50+ | 52% polyester, 48% nylon | 

/opt/homebrew/lib/python3.11/site-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(



> Finished chain.
Response: {'query': 'Please suggest three shirts with sunblocking and tell me why. Give this back to me in markdown code as a table', 'result': 'This document does not provide enough information to suggest three shirts with sunblocking. '}
The time of execution of above program is : 2917.702ms


TESTING
:<__main__.ResultsData object at 0x1569891d0>


> Entering new RetrievalQA chain...

> Finished chain.
Response: {'query': 'Please suggest three shirts with sunblocking and tell me why. Give this back to me in markdown code as a table, with a summary below outlining why sunblocking is important', 'result': "| Shirt Name | Sun Protection Rating | Fabric | Additional Features |\n| --- | --- | --- | --- |\n| Sun Shield Shirt | UPF 50+ | 78% nylon, 22% Lycra Xtra Life fiber | Wicks moisture, abrasion-resistant, fits over swimsuit |\n| Men's Plaid Tropic Shirt | UPF 50+ | 52% polyester, 48% nylon | Wrinkle-free, front and back cape venting, two front bellows pockets |\n| M

/opt/homebrew/lib/python3.11/site-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(



> Finished chain.
Response: {'query': 'Please suggest three shirts with sunblocking and tell me why. Give this back to me in markdown code as a table, with a summary below outlining why sunblocking is important', 'result': "This document only provides information about one specific shirt and does not suggest any other shirts with sunblocking. However, it does mention that the shirt is rated UPF 50+ for superior protection from the sun's UV rays and provides SPF 50+ sun protection, blocking 98% of the sun's harmful rays. Sunblocking is important because it helps to prevent skin damage and reduce the risk of skin cancer caused by exposure to the sun's harmful UV rays. "}
The time of execution of above program is : 4974.671ms


TESTING
:<__main__.ResultsData object at 0x1569891d0>


In [ ]:
    # EXPERIMENTAL LLM QA GEN AND EVALUATION
for strat in strategies:
        # Generate evaluation Q&As
        tuple = generate_qas(file_path, db, llm, strat) #TODO: change this?
        qa = tuple[0]
        examples = tuple[1]

        # Evaluate 
        results_data = evaluate(strat, qa, examples, llm, results_data)


In [62]:
def results_data_to_markdown_table(results_data_list):
    headers = ["Chain Type", "Eval Time", "Tokens Used", "Example Number", "Predicted Query", "Predicted Answer", "Answer", "Result"]
    markdown_table = "| " + " | ".join(headers) + " |\n"
    markdown_table += "| " + " | ".join(["---"] * len(headers)) + " |\n"

    for data in results_data_list:
        for eval in data.eval:
            # Ensure every value is a string, handling None and ensuring dict values are properly formatted or avoided
            row = [
                data.chain_type,
                str(eval.get("time", "")),
                str(eval.get("tokens_used", "")),
                str(eval.get("example_number", "")),
                eval.get("query", ""),
                eval.get("predicted_answer", "") if eval.get("predicted_answer") is not None else "",
                eval.get("answer", ""),
                eval.get("result", "") if eval.get("result") is not None else ""
            ]
            markdown_table += "| " + " | ".join([str(item) for item in row]) + " |\n"
    
    return markdown_table


In [63]:
    # Generate results in markdown
md_table = results_data_to_markdown_table(results_data)

In [68]:
def write_markdown_table_to_file(markdown_table, filename):
    # Write the markdown table to the specified file
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(markdown_table)
    
    print(f"Markdown table successfully written to {filename}")

In [69]:
    # Write results to file
write_markdown_table_to_file(md_table, "results.md")

Markdown table successfully written to results.md
